### Setup

In [6]:
from deap import base, creator, tools
import random
import string
import itertools

signals = list(string.ascii_lowercase)  # Characters used to make words
meanings = list(range(0, 1000))  # 1000 possible meanings

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_word(min_length=1, max_length=3): #defines lenght of word
    length = random.randint(min_length, max_length)
    return ''.join(random.choice(signals) for _ in range(length))

def create_individual():
    individual = creator.Individual()
    individual.local_state = random.choice(meanings)
    individual.age = 0
    for meaning in meanings:

        word = create_word()
        individual[meaning] = word

    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


c:\Users\daalm\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\daalm\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### Evaluation

In [7]:
GLOBAL_STATE = random.choice(meanings)

"""
Adds newly heard word to individual's vocabulary 
if a randomly guessed meaning matches that of the 
speaker.
"""
def learn(individual, new_word, speakers_meaning):
    random_meaning = random.choice(meanings)
    if random_meaning == speakers_meaning:
        individual[random_meaning] = new_word

def pairwise_communication(speaker, listener, GLOBAL_STATE):
    fitness = 0
    # speaker encodes signals
    local_meaning = speaker.local_state
    global_meaning = GLOBAL_STATE

    semantic_local = speaker[local_meaning]
    semantic_global = speaker[global_meaning]

    # only update if listener has words in dictionary
    decoded_local = -100
    decoded_global = -101

    localWordPresentinListener = 0
    globalWordPresentinListener = 0
    # listener decodes local signal or possibly learns new word
    for meaning in meanings:
        if semantic_local == listener[meaning]:  
            decoded_local = meaning
            localWordPresentinListener = 1
        if semantic_global == listener[meaning]:
            decoded_global = meaning
            globalWordPresentinListener = 1

    if localWordPresentinListener == 1:
        if (random.random() < 1):
            learn(listener, semantic_local, local_meaning)

    if globalWordPresentinListener == 1:
        if (random.random() < 1): 
            learn(listener, semantic_global, global_meaning)

    if (decoded_local == local_meaning) & (localWordPresentinListener == 1):
        fitness += 1
    if (decoded_global == global_meaning) & (globalWordPresentinListener == 1):
        fitness += 1

    # penalty if different ideas are represented by the same word
    if local_meaning != global_meaning:
        if semantic_global == semantic_local:
            fitness -= 1
        if decoded_local == decoded_global:
            fitness -= 1
        
    return fitness   
            
def survival_task(individual, GROUP_SIZE):
    penalty = 0
    for i in range(2 * (GROUP_SIZE - 1)):
        meaning1, meaning2 = random.sample(meanings, 2) # two different meanings
        if individual[meaning1] == individual[meaning2]: # same word
            penalty += 2

    return penalty

# evaluate group
def evaluate_group(group, GROUP_SIZE):
    GLOBAL_STATE = random.choice(meanings)
    # reset fitness
    for ind in group:
        # ind.fitness.values = (0,)  # reset fitness
        ind.local_state = random.choice(meanings)

    # speak
    for speaker, listener in itertools.permutations(group, 2):
        fitness_bonus = pairwise_communication(speaker, listener, GLOBAL_STATE)
        speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

    # individual survival
    for ind in group:
        penalty = survival_task(ind, GROUP_SIZE)
        ind.fitness.values = (ind.fitness.values[0] - penalty,)


### Declare population + initial eval

In [8]:
# create pop
population = toolbox.population(n=100)

for ind in population:
    ind.fitness.values = (0,)

toolbox.register("evaluate", evaluate_group)

GROUP_SIZE = 10

# for i in range(500):
groups = [population[i:i + GROUP_SIZE] for i in range(0, len(population), GROUP_SIZE)]
for group in groups:
    toolbox.evaluate(group, GROUP_SIZE)

count = 0
for ind in population:
    if ind.fitness.values[0] > 0:
        print(ind, ind.fitness.values[0])
        count += 1
print(count)

{0: 'zam', 1: 'r', 2: 'gwk', 3: 'q', 4: 'x', 5: 'kbo', 6: 'fx', 7: 'w', 8: 'q', 9: 'mt', 10: 'o', 11: 'pe', 12: 'rw', 13: 'kf', 14: 'sqc', 15: 'fzy', 16: 'h', 17: 'zoa', 18: 'yo', 19: 'yb', 20: 'vhw', 21: 'pvl', 22: 'd', 23: 'a', 24: 'hwo', 25: 'gi', 26: 'xe', 27: 'd', 28: 'gs', 29: 'p', 30: 'm', 31: 'fd', 32: 'in', 33: 'bii', 34: 'y', 35: 'v', 36: 'p', 37: 'fb', 38: 't', 39: 'jlg', 40: 'e', 41: 'n', 42: 'k', 43: 'fs', 44: 'pbo', 45: 'x', 46: 'cr', 47: 'o', 48: 'q', 49: 'zwu', 50: 'fz', 51: 'p', 52: 'i', 53: 'vag', 54: 'w', 55: 'aaq', 56: 'gq', 57: 'w', 58: 'nly', 59: 'ht', 60: 'uj', 61: 'yei', 62: 'y', 63: 'ill', 64: 'ftr', 65: 'y', 66: 'cv', 67: 'li', 68: 'mif', 69: 'ueu', 70: 'q', 71: 'p', 72: 'f', 73: 'c', 74: 'yh', 75: 'ze', 76: 'i', 77: 'ds', 78: 'qws', 79: 'o', 80: 'zh', 81: 'x', 82: 'oai', 83: 'kz', 84: 'kxd', 85: 'ise', 86: 'vj', 87: 'zf', 88: 'a', 89: 'e', 90: 'a', 91: 'tm', 92: 'icx', 93: 'bc', 94: 'lk', 95: 'ec', 96: 'khd', 97: 'fj', 98: 'r', 99: 'gg', 100: 'l', 101: 'bf', 

### Params

In [9]:
from deap import algorithms
import numpy

# Parameters
NUM_GENERATIONS = 100
CXPB = 0.5  # Crossover probability
MUTPB = 0.01  # Mutation probability (seems better at 0 for now or .01)

# mut for signals
def mutateInd(individual, indpb):
    for i in range(len(meanings)):
        if random.random() < indpb:
            individual[i] = create_word()

    return individual,

# tools
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", mutateInd, indpb=0.1)
toolbox.register("select", tools.selRoulette) # inspired by paper
toolbox.register("tourn", tools.selTournament)
toolbox.register("randsel", tools.selRandom)
toolbox.register("stochastic_select", tools.selStochasticUniversalSampling)



stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

# for steady state
def select_next_generation(parents, offspring, max_age=3):
    combined = parents + offspring
    filtered = [ind for ind in combined if ind.age < max_age]
    selected = toolbox.tourn(filtered, len(population), 2)
    return selected



### run algorithm

In [10]:
for gen in range(NUM_GENERATIONS):
    
    #for ES:
    MU = 100
    LAMBDA = 150

    """
    Mating selection
    """
    #offspring = toolbox.randsel(population, LAMBDA)
    #offspring = toolbox.select(population, len(population))
    #offspring = toolbox.tourn(population, len(population), tournsize=2)
    offspring = toolbox.randsel(population, len(population)) # random sel

    offspring = list(map(toolbox.clone, offspring))

    # mutation and crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    """
    Evaluate Fitness
    """
    random.shuffle(offspring)
    for ind in offspring:
        ind.fitness.values = (0,)
    groups = [offspring[i:i + GROUP_SIZE] for i in range(0, len(offspring), GROUP_SIZE)]
    for group in groups:
        toolbox.evaluate(group, GROUP_SIZE)

    for ind in population:
        ind.age += 1

    """
    Survivor Selection
    """
    # ES:
    #population[:] = toolbox.stochastic_select(offspring, MU)

    # steady state:
    population [:] = select_next_generation(population, offspring) 

    # generational:
    #population[:] = offspring


    fits = [ind.fitness.values[0] for ind in population]


    from collections import Counter

    def count_duplicate_values(individual):
        value_count = Counter(individual.values())
        duplicates = [value for value, count in value_count.items() if count > 1]
        return len(duplicates)  # Return the number of unique values that are duplicates

    dup_list = []

    for individual in population:
        num_duplicates = count_duplicate_values(individual)
        dup_list.append(num_duplicates)
       
    sum_dups = 0
    for dup in dup_list:
        sum_dups += dup
    #print("Dup Avg:", sum_dups / len(ind) / len(population))



    length = len(population)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"Generation {gen}: Min {min(fits)}, Max {max(fits)}, Avg {mean}, Std {std}, Avg % of duplicate genes across participants {sum_dups / len(ind) / len(population)}")

# print final pop
for ind in population:
    print(ind, ind.fitness.values)

Generation 0: Min 0.0, Max 5.0, Avg 0.31, Std 0.8797158632194829, Avg % of duplicate genes across participants 0.0892
Generation 1: Min 0.0, Max 7.0, Avg 0.74, Std 1.1629273408085303, Avg % of duplicate genes across participants 0.08986000000000001
Generation 2: Min -2.0, Max 4.0, Avg 1.18, Std 1.23596116443843, Avg % of duplicate genes across participants 0.08865
Generation 3: Min -1.0, Max 10.0, Avg 2.54, Std 2.2865694828716663, Avg % of duplicate genes across participants 0.08897000000000001
Generation 4: Min 0.0, Max 16.0, Avg 5.42, Std 4.18373039284321, Avg % of duplicate genes across participants 0.08839000000000001
Generation 5: Min 0.0, Max 16.0, Avg 6.21, Std 4.271521977000704, Avg % of duplicate genes across participants 0.08924
Generation 6: Min 0.0, Max 17.0, Avg 6.34, Std 4.290034964892477, Avg % of duplicate genes across participants 0.09049
Generation 7: Min 0.0, Max 21.0, Avg 8.2, Std 5.36283507111677, Avg % of duplicate genes across participants 0.08942
Generation 8: M

### Final population

In [11]:
for ind in population:
    print(ind)

{0: 'cir', 1: 'nf', 2: 'huj', 3: 'ka', 4: 'b', 5: 'db', 6: 'ln', 7: 'j', 8: 't', 9: 't', 10: 'xzt', 11: 'yz', 12: 'wnd', 13: 'h', 14: 'u', 15: 'z', 16: 'xrq', 17: 'grg', 18: 'z', 19: 'cet', 20: 'a', 21: 'ax', 22: 'z', 23: 'bh', 24: 'r', 25: 'g', 26: 'sm', 27: 'kc', 28: 'yu', 29: 'hcr', 30: 'op', 31: 'co', 32: 'ru', 33: 'i', 34: 'yhp', 35: 'mj', 36: 'sw', 37: 'f', 38: 'c', 39: 'twg', 40: 'h', 41: 'o', 42: 'ny', 43: 'ci', 44: 'd', 45: 'd', 46: 'ar', 47: 'x', 48: 'ggc', 49: 'kx', 50: 'dgi', 51: 'p', 52: 'fpj', 53: 'oeu', 54: 'gk', 55: 'e', 56: 'j', 57: 'und', 58: 'r', 59: 'w', 60: 'a', 61: 'vyp', 62: 'm', 63: 'wn', 64: 'p', 65: 'ew', 66: 'n', 67: 'ugt', 68: 'vw', 69: 'i', 70: 'q', 71: 'jjz', 72: 'o', 73: 'b', 74: 'l', 75: 'fcn', 76: 'i', 77: 'z', 78: 'vt', 79: 'vpg', 80: 'njt', 81: 'db', 82: 'h', 83: 'ura', 84: 'g', 85: 'mi', 86: 'tpi', 87: 'btn', 88: 'c', 89: 'sx', 90: 'bgf', 91: 'mve', 92: 'gq', 93: 'top', 94: 'r', 95: 'wfe', 96: 'oqd', 97: 'jp', 98: 'aq', 99: 'bjm', 100: 'oxm', 101: 'n